In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import pandas as pd

# Load model once at startup
model = joblib.load("final_avoidable_ed_model.joblib")

# Extract raw input feature names from the preprocessing step
input_features = model.named_steps["preprocess"].feature_names_in_

# Dynamically define request schema
class EDVisitInput(BaseModel):
    __annotations__ = {feat: float for feat in input_features}

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Avoidable ED Prediction API is running"}

@app.post("/predict")
def predict(data: EDVisitInput):
    df = pd.DataFrame([data.dict()])
    pred = model.predict(df)[0]
    prob = model.predict_proba(df)[0][1]
    return {"prediction": int(pred), "probability": float(prob)}